In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd
import mysql.connector
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [2]:
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql
db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  


In [4]:
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [5]:
df_sqlcsv = pd.read_csv('reviews_price_btc.csv')
df_sqlcsv.time = pd.to_datetime(df_sqlcsv.time)
df_sqlcsv["price_btc_reviews_satoshi"] = df_sqlcsv["price_btc_review"] * 100000000
df_sqlcsv = df_sqlcsv.sort_values(by="time",ascending=True)

In [6]:
query = """select a.vendor_id from (
SELECT vendor_id, count(*) as count
from vendors 
group by vendor_id
) as a where a.count = 2;
"""
df = pd.read_sql(query, con=db_connection)
df_sqlcsv = df_sqlcsv[df_sqlcsv["vendor_id"].isin(df["vendor_id"])]
df_sqlcsv.head(3)

,Unnamed: 0,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
30502,30502,13.31,2017-09-08 11:24:25,aj [3],e0faa2e54e023f7431cd9c473ae40716,109299,0.003633,3.632543e+05
30501,30501,13.07,2017-09-12 08:49:17,T**e [2],e0faa2e54e023f7431cd9c473ae40716,109298,0.003745,3.744986e+05
26608,26608,165.14,2017-10-09 14:35:07,tk [5],a081033ab42c85ae4101549cd64ff4cb,105344,0.040876,4.087624e+06


In [21]:
len(df_sqlcsv)

9057

In [7]:
# df_neo4jbig = pd.DataFrame(columns=['date', 'inDegree', 'inSum', 'outDegree', 'outSum', 'txid','vendor'])
df_neo4jbiga = pd.DataFrame(columns=['inDegree', 'outDegree', 'address','vendor'])
for i in df_sqlcsv.head(10).itertuples():
    neo_results = []
    neo_results2 = []
    start = str(i.time.date()- timedelta(days=3))
    end = str(i.time.date())
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    neo_query = session.run("""
    WITH '""" + start + """' as minDate,  '""" + end + """' as maxDate, """
    + minSum + """ as minSum
    MATCH (t:Transaction) 
        WHERE  t.inDegree = 1 and ( t.outDegree = 2 OR t.outDegree = 1 ) AND date(minDate) <= t.date <= date(maxDate) 
        and minSum <= t.inSum <= minSum * 1.10
    WITH t, minSum, minDate, maxDate
    MATCH (t)-[r:RECEIVES]->(a:Address),
        (t)-[r1:RECEIVES]->(c:Address)-[s:SENDS]->(t)
        WHERE minSum  <= r.value <= minSum * 1.05 AND c <> a and a.inDegree <> 2
    WITH a, r, minDate, maxDate
    MATCH (a)-[s:SENDS]->(mix:Transaction)
        WHERE date(minDate) <= mix.date <= date(maxDate) AND r.value >= s.value 
    RETURN a, r.value
    """)
    for r in neo_query:
#         neo_results.append(r[0])
        neo_results2.append(r[0])
        neo_results.append(r[1])
#     df_neo4j = pd.DataFrame(neo_results)
#     df_neo4j["vendor"] = i.vendor_id
#     df_neo4j["review"] = i.vendor_review_id
#     df_neo4jbig = pd.concat([df_neo4jbig,df_neo4j])
    
    df_neo4j2 = pd.DataFrame(neo_results2)
    df_neo4j2["vendor"] = i.vendor_id
    df_neo4j2["review"] = i.vendor_review_id
    df_neo4j2["rvalue"] = pd.Series(neo_results)
    df_neo4jbiga = pd.concat([df_neo4jbiga,df_neo4j2])
    
# display(df_neo4jbig.head() )
display(df_neo4jbiga.head() )

,inDegree,outDegree,address,vendor,maxDate,minDate,totalMoneyReceived,review,rvalue
0,7,7,1NDbc6X1Sixz1xsYaciUei5potr7qYNqmF,e0faa2e54e023f7431cd9c473ae40716,2017-12-18,2017-09-05,2169451.0,109299.0,375808.0
1,1,1,1H1VYroTosiT9K7rSbM5WafqvcjH72upUm,e0faa2e54e023f7431cd9c473ae40716,2017-09-05,2017-09-05,374544.0,109299.0,374544.0
2,1,1,12guWqmKwkP4Xb3J6TcRDnbJtx1ZVdHYVo,e0faa2e54e023f7431cd9c473ae40716,2017-09-07,2017-09-07,363300.0,109299.0,363300.0
3,1,1,14ZHx8dFJJDg8DUi5Ttdm7bgKyxhsjTQ6F,e0faa2e54e023f7431cd9c473ae40716,2017-09-08,2017-09-08,372297.0,109299.0,372297.0
4,15,15,16gVj2JEPrT51neYGkmUUBE1pm9zREQNuZ,e0faa2e54e023f7431cd9c473ae40716,2017-11-04,2017-09-03,4098000.0,109299.0,370000.0


In [8]:
i

Pandas(Index=30080, _1=30080, price_eur=25.12, time=Timestamp('2017-10-20 13:45:51'), customer=' r**3 [4]', vendor_id='cd6ef672c2ff60a03fd009ec48602c0a', vendor_review_id=108868, price_btc_review=0.0050695243284696575, price_btc_reviews_satoshi=506952.43284696573)

In [9]:
# df_neo4jbig.to_csv("QueryResultsTransaction1Mix.csv")
df_neo4jbiga.to_csv("QueryResultsTransaction1EscrowActiveOnBoth.csv")

In [19]:
df_neo4jbiga.groupby("review").count()

,inDegree,outDegree,address,vendor,maxDate,minDate,totalMoneyReceived,rvalue
review,,,,,,,,
104261.0,66308,66308,66308,66308,66308,66308,66308,66308
105343.0,161425,161425,161425,161425,161425,161425,161425,161425
105344.0,7074,7074,7074,7074,7074,7074,7074,7074
108868.0,432,432,432,432,432,432,432,432
108869.0,107482,107482,107482,107482,107482,107482,107482,107482
108870.0,5929,5929,5929,5929,5929,5929,5929,5929
108871.0,5929,5929,5929,5929,5929,5929,5929,5929
108872.0,259,259,259,259,259,259,259,259
109298.0,5,5,5,5,5,5,5,5


In [18]:
df_sqlcsv.head(10)

,Unnamed: 0,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
30502,30502,13.31,2017-09-08 11:24:25,aj [3],e0faa2e54e023f7431cd9c473ae40716,109299,0.003633,3.632543e+05
30501,30501,13.07,2017-09-12 08:49:17,T**e [2],e0faa2e54e023f7431cd9c473ae40716,109298,0.003745,3.744986e+05
26608,26608,165.14,2017-10-09 14:35:07,tk [5],a081033ab42c85ae4101549cd64ff4cb,105344,0.040876,4.087624e+06
30084,30084,24.62,2017-10-17 09:27:02,rk [5],cd6ef672c2ff60a03fd009ec48602c0a,108872,0.005179,5.178688e+05
30083,30083,58.78,2017-10-17 11:40:32,f**1 [4],cd6ef672c2ff60a03fd009ec48602c0a,108871,0.012364,1.236406e+06
30082,30082,58.82,2017-10-17 19:00:05,O**d [2],cd6ef672c2ff60a03fd009ec48602c0a,108870,0.012372,1.237248e+06
30081,30081,77.28,2017-10-19 18:29:38,O**d [2],cd6ef672c2ff60a03fd009ec48602c0a,108869,0.016077,1.607722e+06
26607,26607,401.17,2017-10-19 19:25:48,tk [5],a081033ab42c85ae4101549cd64ff4cb,105343,0.083459,8.345885e+06
25551,25551,277.69,2017-10-20 13:31:27,I***5 [2],8438ba1a66f83580db67292ecced8f9c,104261,0.056041,5.604125e+06
30080,30080,25.12,2017-10-20 13:45:51,r**3 [4],cd6ef672c2ff60a03fd009ec48602c0a,108868,0.005070,5.069524e+05


In [10]:
len(df_neo4jbiga)

354848

In [12]:
session.close()